In [1]:
from crewai import Agent, Crew, Task

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
# Initialize the tools
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults()
scrape_tool = ScrapeWebsiteTool()

In [4]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

In [9]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [5]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

In [14]:
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

In [15]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'RELIANCE',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [16]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (RELIANCE). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for RELIANCE stock to the Data Analyst. They will need to use statistical modeling and machine learning to identify trends and predict market movements.

Action:
Delegate work to co-worker

Action Input:
{
    "co-worker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for RELIANCE stock. Use statistical modeling and machine learning to identify trends and predict market movements.",
    "context": "This task requires a deep understanding of market data analysis, statistical modeling, and machine learning techniques. The goal is to provide insights and alerts about significant market opportunities or thr

In [17]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for RELIANCE:

Introduction:
The proposed trading strategy for RELIANCE is based on a momentum-based day trading approach, aiming to capitalize on positive market outlook and potential growth opportunities in the renewable energy sector. This report provides a comprehensive analysis of potential risks associated with the trading strategies and execution plans for RELIANCE, along with suggested mitigation strategies.

Risk Analysis:
1. Market Risk: The stock market is subject to volatility and uncertainties in market trends, which can impact the performance of the trading strategy. To mitigate this risk, it is essential to closely monitor market news and trends to make informed decisions.

2. Liquidity Risk: RELIANCE appears to be highly liquid, which reduces the liquidity risk associated with trading the stock. This factor provides flexibility in executing trades and managing positions effectively.

3. Operational Risk: Operational risks such as the loss of share certificates or technical issues can disrupt trading activities. To address this risk, proper management and control procedures should be in place to ensure smooth operations.

Mitigation Strategies:
1. Monitor Market News and Trends: Stay informed about any developments or news that could impact the stock price of RELIANCE. Make adjustments to the trading strategy based on market conditions.

2. Use Stop-Loss Orders: Implement stop-loss orders to limit potential losses and protect capital in case the market moves against the trade.

3. Diversify Trades: Consider diversifying trades among a variety of stocks in the renewable energy sector to spread risk and optimize returns.

4. Adjust Strategy as Needed: Continuously evaluate the effectiveness of the trading strategy and make adjustments as necessary to adapt to changing market conditions.

Conclusion:
By effectively managing the identified risks and implementing the suggested mitigation strategies, the trading team can enhance the performance of the momentum-based day trading strategy for RELIANCE. It is crucial to maintain discipline, monitor market conditions closely, and be prepared to make informed decisions to achieve successful outcomes in trading RELIANCE.